<a href="https://colab.research.google.com/github/bsenst/capstone1-skin-lesion-classifier/blob/main/load_h5_model_and_deploy_with_gradio_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/08-deep-learning/notebook.ipynb

The model was trained as described in this kaggle notebook: https://www.kaggle.com/bnzn261029/capstone1-ham10k-skincancer

*Disclaimer: This work is part of an educational project. It is not intended for clinical application. As such it can not make real world predictions for skin lesions. To get recommendations regarding skin lesions one should ask for expert advice such as provided by a dermatologist.*

# setting up libraries

In [1]:
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 13.8 MB/s 


In [2]:
from huggingface_hub import login
# huggingface account and access token is necessary
login()

In [3]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 17.5 MB/s 
     |████████████████████████████████| 7.6 MB 48.4 MB/s 


In [4]:
from transformers import ImageClassificationPipeline
from transformers import TFAutoModel
from transformers import TFPreTrainedModel
import huggingface_hub

# load model from huggingface hub

In [5]:
from huggingface_hub import Repository
repo = Repository(local_dir="repo", clone_from="bsenst/skin-cancer-HAM10k")

Cloning https://huggingface.co/bsenst/skin-cancer-HAM10k into local empty directory.


Download file xception_v4_1_07_0.699.h5:   0%|          | 1.43k/80.0M [00:00<?, ?B/s]

Clean file xception_v4_1_07_0.699.h5:   0%|          | 1.00k/80.0M [00:00<?, ?B/s]

In [6]:
import keras
model = keras.models.load_model("/content/repo/xception_v4_1_07_0.699.h5")

# predict on example image

In [8]:
from tensorflow.keras.preprocessing.image import load_img
! wget "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Melanoma.jpg/320px-Melanoma.jpg"
img = load_img("320px-Melanoma.jpg", target_size=(150, 150))
import numpy as np
x = np.array(img)
from tensorflow.keras.applications.xception import preprocess_input
X = preprocess_input(X)
pred = model.predict(X)
classes = [
    "akiec",
    "bcc",
    "bkl",
    "df",
    "mel",
    "nv",
    "vasc",
]
dict(zip(classes, pred[0]))

--2022-12-01 19:48:22--  https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Melanoma.jpg/320px-Melanoma.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 91.198.174.208, 2620:0:862:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|91.198.174.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25837 (25K) [image/jpeg]
Saving to: ‘320px-Melanoma.jpg’

320px-Melanoma.jpg  100%[===================>]  25.23K  --.-KB/s    in 0.004s  

2022-12-01 19:48:22 (7.02 MB/s) - ‘320px-Melanoma.jpg’ saved [25837/25837]

1/1 [==============================] - 1s 1s/step


{'akiec': -19.072325,
 'bcc': -18.326487,
 'bkl': -8.601684,
 'df': -34.96761,
 'mel': 7.388905,
 'nv': 2.8025546,
 'vasc': -5.3949637}

In [9]:
! pip install gradio
import gradio as gr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.6 MB 14.8 MB/s 
     |████████████████████████████████| 106 kB 58.8 MB/s 
     |████████████████████████████████| 213 kB 54.7 MB/s 
     |████████████████████████████████| 55 kB 4.3 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 278 kB 66.4 MB/s 
     |████████████████████████████████| 2.3 MB 55.1 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 68 kB 6.6 MB/s 
     |████████████████████████████████| 68 kB 6.9 MB/s 
     |████████████████████████████████| 68 kB 6.4 MB/s 
     |████████████████████████████████| 68 kB 6.9 

# deploy as gradio app

In [10]:
def classify_image(inp):
  img = load_img(inp, target_size=(150, 150))
  x = np.array(img)
  X = np.array([x])
  X = preprocess_input(X)
  pred = model.predict(X).flatten()
  # return {classes[i]: float(pred[i]) for i in range(7)}
  # return {'cat': 0.3, 'dog': 0.7}

"""gr.Interface(
    fn=classify_image, 
    inputs=gr.Image(),
    outputs=gr.outputs.Label(num_top_classes=4),
    examples=["320px-Melanoma.jpg"],
).launch()"""

def image_classifier(inp):
    x = np.array(img)
    X = np.array([x])
    X = preprocess_input(X)
    pred = model.predict(X).flatten()
    # return {classes[i]: float([0.3,0.4,0.6,0.2,0.9,0.1,0.5][i]) for i in range(7)}
    return {classes[i]: float(pred[i]) for i in range(7)}

demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label", examples=["320px-Melanoma.jpg"],)
demo.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# check dependencies

In [11]:
gr.__version__

'3.12.0'

In [15]:
keras.__version__

'2.9.0'

In [17]:
import transformers
transformers.__version__

'4.24.0'

In [19]:
import huggingface_hub
huggingface_hub.__version__

'0.11.1'